In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
# from video_swin_transformer import SwinTransformer3D
from R2plus1d_video_swin_transformer import SwinTransformer3D, R2plus1d_backbone
from einops import rearrange, reduce
from tqdm import tqdm
import gc
import copy

# PATH = '/home/ssrlab/kw/개성형성/video_swin_transformer'
# os.chdir(PATH)



logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./r2plus1d_video_swin_transformer/LOG/Log_.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
save_model_file_path = './16x224x224_save_model/{}_{}.{}'

/home/ssrlab/anaconda3/envs/kyuwon_video_swin_transformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#model save path
save_model_file_path = './save_swintransformer_folder/{}_{}.{}'

with open('./train_15frame_facedetect_sum.dat', "rb") as training_file:
    train_set_data = pickle.load(training_file)
with open('./train_15frame_facedetect_label.dat', "rb") as training_file:
    train_y = pickle.load(training_file)
with open('./valid_15frame_facedetect_sum.dat', "rb") as training_file:
    valid_set_data = pickle.load(training_file)
with open('./valid_15frame_facedetect_label.dat', "rb") as training_file:
    valid_y = pickle.load(training_file)
with open('./test_15frame_facedetect_sum.dat', "rb") as training_file:
    test_set_data = pickle.load(training_file)
with open('./test_15frame_facedetect_label.dat', "rb") as training_file:
    test_y = pickle.load(training_file)


train_set_data = np.array(train_set_data)
test_set_data = np.array(test_set_data)
valid_set_data = np.array(valid_set_data)

train_y = np.array(train_y)
test_y  = np.array(test_y)
valid_y = np.array(valid_y)

In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [5]:
class ChalearnDataset(Dataset):
    def __init__(self,imagedata,tagdata,transform=None):
        self.imagedata=imagedata
        self.tagdata=tagdata
        self.transform = transform
        
    def __len__(self):
        return len(self.imagedata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_data=(self.imagedata[idx][::2] * 255 )
        image_data=torch.FloatTensor(image_data)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return image_data,big_five_sorces

In [6]:
logger.debug('=============Video swin transformerTrain Strat!=============')
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:1' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)
    

[2023-08-24 20:43:28,519]::2254233699::DEBUG::=============Video swin transformerTrain Strat!=============


True
학습을 진행하는 기기: cuda:1


In [7]:
batchsz = 8
num_workerssz = 5
lr = 3e-5
epochs = 120

#device = torch.device('cuda')
train_dataset = ChalearnDataset(imagedata=train_set_data,tagdata=train_y,transform=transform)
val_dataset = ChalearnDataset(imagedata=valid_set_data,tagdata=valid_y,transform=transform)
test_dataset = ChalearnDataset(imagedata=test_set_data,tagdata=test_y,transform=transform)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0

In [9]:
model=SwinTransformer3D()

model.to(device)
lr = 3e-05
criterion = torch.nn.L1Loss().to(device)  # 손실함수 MAE
# optimizer = torch.optim.Adam(model.parameters(), lr=lr) # 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

start_epoch = 0

/home/ssrlab/anaconda3/envs/kyuwon_video_swin_transformer/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [11]:
from torchinfo import summary

summary(model, input_size = (8,3,15,224,224), col_names = ['input_size','output_size','num_params'])

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #
SwinTransformer3D                                       [8, 3, 15, 224, 224]      [8, 5]                    --
├─R2plus1d_backbone: 1-1                                [8, 3, 15, 224, 224]      [8, 256, 4, 28, 28]       --
│    └─VideoResNet: 2-1                                 [8, 3, 15, 112, 112]      [8, 256, 4, 14, 14]       --
│    │    └─R2Plus1dStem: 3-1                           [8, 3, 15, 112, 112]      [8, 64, 15, 56, 56]       15,473
│    │    └─Sequential: 3-2                             [8, 64, 15, 56, 56]       [8, 64, 15, 56, 56]       666,048
│    │    └─Sequential: 3-3                             [8, 64, 15, 56, 56]       [8, 128, 8, 28, 28]       3,332,364
│    │    └─Sequential: 3-4                             [8, 128, 8, 28, 28]       [8, 256, 4, 14, 14]       20,400,408
│    └─VideoResNet: 2-2                                 [8, 3, 15, 112, 112]      [

In [12]:
print(model)

SwinTransformer3D(
  (r2plus1d_backbone): R2plus1d_backbone(
    (model): VideoResNet(
      (stem): R2Plus1dStem(
        (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        (1): BatchNorm3d(45, eps=0.001, momentum=0.9, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        (4): BatchNorm3d(64, eps=0.001, momentum=0.9, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Sequential(
            (0): Conv2Plus1D(
              (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
              (1): BatchNorm3d(144, eps=0.001, momentum=0.9, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
              (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 

In [ ]:
with torch.cuda.device(1):
    
    trainingEpoch_loss = []
    validationEpoch_loss = []
    
    for i in range(epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        
        model.train()
        for image_data, big_five_data in tqdm(train_dataloader):

            image_data = rearrange(image_data, 'b d h w c -> b c d h w')
            image_data = image_data.to(device)

            big_five_data = reduce(big_five_data,'b c d -> b c', 'max')
            big_five_data = big_five_data.to(device)
            
            optimizer.zero_grad()  # 기울기가 0이 됩니다.

            hypothesis = model(image_data)  # 모델의 예측 결과를 저장합니다.

            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            
            loss.backward()  # 역방향 전파입니다.
            optimizer.step()  # 매개 변수를 업데이트합니다
            
            train_avg_loss += loss  # 훈련 손실의 평균치입니다
        train_avg_loss=train_avg_loss/len(train_dataloader)
        trainingEpoch_loss.append(train_avg_loss)
        print('Epoch = {}, loss = {}'.format(i+1,train_avg_loss))
        
        
        with torch.no_grad():#validate
            model.eval()
            for image_data, big_five_data in tqdm(val_dataloader):
                
                image_data = rearrange(image_data, 'b d h w c -> b c d h w')
                image_data = image_data.to(device)
                
                big_five_data = reduce(big_five_data,'b c d -> b c', 'max')
                big_five_data = big_five_data.to(device)
                
                hypothesis = model(image_data)
                
                val_loss = criterion(hypothesis, big_five_data)
                val_avg_loss += val_loss
                
            val_avg_loss=val_avg_loss/len(val_dataloader)
            validationEpoch_loss.append(val_avg_loss)
        print('Epoch = {}, val_loss = {}'.format(i+1,val_avg_loss))
        print('\n')

        # torch.cuda.empty_cache()
        
        max_value=val_avg_loss
        start_epoch+=1
        if(i%5 == 0):
            torch.save({
                    'epoch': i+1,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'loss': val_avg_loss,
                }, save_model_file_path.format('model',start_epoch,'pth'))
        logger.debug('Epoch: {} , 1-MAE: {:.4f} , train_loss:{:.4f} , val_loss:{:.4f} , MAX Value:{:.4f} , MAX Value Epoch:{}'.format(i+1, 1-val_avg_loss, train_avg_loss, val_avg_loss, max_value, start_epoch))
        gc.collect()

In [ ]:
# model test

# model=SwinTransformer3D()

# model.to(device)

# criterion = torch.nn.L1Loss().to(device)  # 손실함수 MAE
# optimizer = torch.optim.Adam(model.parameters(), lr=lr) # 옵티마이저


# checkpoint=torch.load("./video_swin_transformer/save_swintransformer_folder/model_42.pth",map_location=device)
# model.load_state_dict(checkpoint["model"])
# optimizer.load_state_dict(checkpoint["optimizer"])

with torch.no_grad():
    model.eval()
    test_avg_loss=0
    for image_data, big_five_data in tqdm(test_dataloader):
        
        
        image_data = rearrange(image_data, 'b d h w c -> b c d h w')
        image_data = image_data.to(device)
                
        big_five_data = reduce(big_five_data,'b c d -> b c', 'max')
        big_five_data = big_five_data.to(device)
                
                
        # image_data=torch.reshape(image_data,(batchsz, 3, 15, 128, 128))
        # image_data = image_data.to(device)

        # big_five_data = torch.reshape(big_five_data, (batchsz, 5))
        # big_five_data = big_five_data.to(device)

        hypothesis = model(image_data)

        test_loss = criterion(hypothesis, big_five_data)
        test_avg_loss += test_loss  # 훈련 손실의 평균치입니다
    test_avg_loss=test_avg_loss/len(test_dataloader)
    print(1 - test_avg_loss)

In [ ]:
temp_1 = []
temp_2 = []
for i in range(len(trainingEpoch_loss)):
    temp_1.append(trainingEpoch_loss[i].item())
    temp_2.append(validationEpoch_loss[i].item())

In [ ]:
from matplotlib import pyplot as plt

plt.clf()
# plt.plot(temp_1, label='train_loss')
plt.plot(temp_2, label='val_loss')
plt.title('Current Model') # 16 224 224 3
plt.xlabel('Epoch')
plt.ylabel('1 - MAE')
plt.legend()
plt.show()

In [ ]:
train_1_mae = []
val_1_mae = []
for i in range(len(trainingEpoch_loss)):
    train_1_mae.append(1-temp_1[i])
    val_1_mae.append(1-temp_2[i])

In [ ]:
from matplotlib import pyplot as plt

plt.clf()
plt.plot(val_1_mae, label='val_loss')
plt.plot(train_1_mae, label='train_loss')
plt.title('Current Model')
plt.xlabel('Epoch')
plt.ylabel('1 - MAE')
plt.legend()
plt.show()

In [ ]:
plt.clf()
plt.plot(val_1_mae, label='val_loss')
# plt.plot(train_1_mae, label='train_loss')
plt.title('Current Model')
plt.xlabel('Epoch')
plt.ylabel('1 - MAE')
plt.legend()
plt.show()